In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from corelab.core.session import CoreLabSession
lab_session = CoreLabSession('xgboost', 'customer-churn', default_folder='processing_notebook', create_run_folder=True, aws_profile='sagemaker-role')
lab_session.print()
core_session = lab_session.core_session

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/machiel/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name machiel-crystalline to get Role path.


falling back to profile: sagemaker-role
AWS region: eu-central-1
Execution role arn:aws:iam::136548476532:role/service-role/AmazonSageMaker-ExecutionRole-20250902T164316
Output bucket uri: s3://sagemaker-eu-central-1-136548476532/processing_notebook/2025-09-16T08-48-49
Framework: xgboost
Project name: customer-churn


In [3]:
# Complete sagemaker-core imports
from sagemaker_core.main.shapes import (
    AppSpecification, 
    ProcessingResources, 
    ProcessingInput, 
    ProcessingClusterConfig,
    ProcessingOutput,
    ProcessingOutputConfig,
    ProcessingStoppingCondition,
    ProcessingS3Input,
    ProcessingS3Output
)
from sagemaker_core.main.resources import ProcessingJob
import time

print("All sagemaker-core modules imported successfully")

All sagemaker-core modules imported successfully


In [8]:
def upload_code_and_data():
    code = lab_session.upload_file('src', 'preprocessing.py', 'prepare-churn-code')
    # Define data and output URIs
    data = f"s3://sagemaker-example-files-prod-{lab_session.region}/datasets/tabular/synthetic/churn.txt"
    output = lab_session.jobs_output_s3_uri

    return code, data, output

upload_code_and_data()

src_path: src/preprocessing.py
dest_path: processing_notebook/2025-09-16T08-48-49/prepare-churn-code/preprocessing.py
full_url: s3://sagemaker-eu-central-1-136548476532/processing_notebook/2025-09-16T08-48-49/prepare-churn-code/preprocessing.py


('s3://sagemaker-eu-central-1-136548476532/processing_notebook/2025-09-16T08-48-49/prepare-churn-code/preprocessing.py',
 's3://sagemaker-example-files-prod-eu-central-1/datasets/tabular/synthetic/churn.txt',
 's3://sagemaker-eu-central-1-136548476532/processing_notebook/2025-09-16T08-48-49/jobs')

In [10]:
# Create and run the Processing Job using sagemaker-core
code_s3_uri, data_s3_uri, output_s3_uri = upload_code_and_data()

print(f"📂 Code S3 URI: {code_s3_uri}")
print(f"📁 Data S3 URI: {data_s3_uri}")
print(f"📤 Output S3 URI: {output_s3_uri}")

job_name = lab_session.processing_job_name
job = ProcessingJob.create(
    processing_job_name=job_name,
    role_arn=lab_session.role,
    session=lab_session.core_session.boto_session,
    region=lab_session.region,
    app_specification=AppSpecification(
        image_uri=lab_session.retrieve_image('1.7-1'),
        container_entrypoint=["python3", "/opt/ml/processing/input/code/preprocessing.py"],
        container_arguments=["--train-test-split", "0.33"]
    ),
    processing_resources=ProcessingResources(
        cluster_config=ProcessingClusterConfig(
            instance_type="ml.m5.large",
            instance_count=1,
            volume_size_in_gb=30
        )
    ),
    processing_inputs=[
        ProcessingInput(
            input_name="code",
            app_managed=False,
            s3_input=ProcessingS3Input(
                s3_uri=code_s3_uri,
                local_path="/opt/ml/processing/input/code",
                s3_data_type="S3Prefix",
                s3_input_mode="File"
            )
        ),
        ProcessingInput(
            input_name="data",
            app_managed=False,
            s3_input=ProcessingS3Input(
                s3_uri=data_s3_uri,
                local_path="/opt/ml/processing/input/data",
                s3_data_type="S3Prefix",
                s3_input_mode="File"
            )
        )
    ],
    processing_output_config=ProcessingOutputConfig(
        outputs=[
            ProcessingOutput(
                output_name="processed",
                app_managed=False,
                s3_output=ProcessingS3Output(
                    s3_uri=output_s3_uri + '/' + job_name,
                    local_path="/opt/ml/processing/output",
                    s3_upload_mode="EndOfJob"
                )
            )
        ]
    ),
    environment={"PYTHONUNBUFFERED": "1"},
    stopping_condition=ProcessingStoppingCondition(
        max_runtime_in_seconds=3600
    )
)

print(f"🚀 Created Processing Job: {job.processing_job_name}")
print(f"📊 Initial Status: {job.processing_job_status}")

job.wait()

src_path: src/preprocessing.py
dest_path: processing_notebook/2025-09-16T08-48-49/prepare-churn-code/preprocessing.py
full_url: s3://sagemaker-eu-central-1-136548476532/processing_notebook/2025-09-16T08-48-49/prepare-churn-code/preprocessing.py
📂 Code S3 URI: s3://sagemaker-eu-central-1-136548476532/processing_notebook/2025-09-16T08-48-49/prepare-churn-code/preprocessing.py
📁 Data S3 URI: s3://sagemaker-example-files-prod-eu-central-1/datasets/tabular/synthetic/churn.txt
📤 Output S3 URI: s3://sagemaker-eu-central-1-136548476532/processing_notebook/2025-09-16T08-48-49/jobs


[09/16/25 11:05:52] INFO     Creating processing_job resource.                                   ]8;id=446442;file:///Users/machiel/Development/crystalline/sagemaker/corelab/.venv/lib/python3.13/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=379040;file:///Users/machiel/Development/crystalline/sagemaker/corelab/.venv/lib/python3.13/site-packages/sagemaker_core/main/resources.py#26174\26174]8;;\

Output()

🚀 Created Processing Job: customer-churn-xgboost-processing-2025-09-16T09-05-52
📊 Initial Status: InProgress


[09/16/25 11:08:25] INFO     Final Resource Status: Completed                                    ]8;id=830675;file:///Users/machiel/Development/crystalline/sagemaker/corelab/.venv/lib/python3.13/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=110123;file:///Users/machiel/Development/crystalline/sagemaker/corelab/.venv/lib/python3.13/site-packages/sagemaker_core/main/resources.py#26386\26386]8;;\